In [15]:
import pandas as pd
import datetime as dt

dataset = pd.read_csv('./data/BTC-Daily.csv')

date_begin = dt.date(2015, 1, 1)
date_end = dt.date(2016, 12, 31)

# convert column 'date' of dataset to datetime format
dataset['date'] = pd.to_datetime(dataset['date'])

# filter dataset by date
dataset = dataset[(dataset['date'] >= date_begin) & (dataset['date'] <= date_end)]

dataset.describe()

TypeError: Invalid comparison between dtype=datetime64[ns] and date

In [ ]:
# make a candlestick plot of the dataset
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=dataset['date'],
                open=dataset['open'],
                high=dataset['high'],
                low=dataset['low'],
                close=dataset['close'])])

fig.show()

In [16]:
# using river, create an ensemble model that predicts the price of bitcoin by using the previous 365 days of data
from river import compose
from river import datasets
from river import ensemble
from river import evaluate
from river import feature_extraction
from river import linear_model
from river import metrics
from river import preprocessing
from river import stats
from river import time_series

model = (
    time_series.Rolling(
        window=365,
        step=1,
        get_features=compose.Select('open', 'high', 'low', 'close')
    ) |
    preprocessing.StandardScaler() |
    ensemble.RandomForestRegressor()
)

# evaluate the model
metric = metrics.MAE()

for x, y in dataset[['open', 'high', 'low', 'close']].iterrows():
    y_pred = model.predict_one(y)
    model = model.learn_one(y)
    metric = metric.update(y, y_pred)

metric

# numpy.ufunc size changed, may indicate binary incompatibility. Expected 232 from C header, got 216 from PyObject
# https://stackoverflow.com/questions/65011252/river-installation-error

# solve river installing problems
# https://stackoverflow.com/questions/65011252/river-installation-error
#   pip install --upgrade pip
#   pip install --upgrade setuptools
#   pip install --upgrade wheel

ValueError: numpy.ufunc size changed, may indicate binary incompatibility. Expected 232 from C header, got 216 from PyObject